# Introduction

# Methods

### RNA Extraction, Library Preparation, and Sequencing

(Going to be written by Dr. Wheeler)

### Read QC and Alignment

To perform QC (quality control) on our sequenced reads, we first performed FASTQC (v. 0.12.1) analysis to compile and score the reads overall. To view these quality scores, multiqc (v. 1.17) was used to view a html file. This allowed us to be given quality scores on the reads for categories: Per base sequence quality, Per tile sequence quality, Per sequence quality score, Per base sequence content, Per sequence GC content, Per base N content, Sequence length distribution, Sequence duplication levels, Overrepresented sequences, and Adapter content. A score was associated for each of the 16 paired intestine and liver miracidia samples. After first evaluation of the scores, we found that there were areas where trimming was necessary to improve the quality of the reads. We limited the reads to be a minimum of 20 base pairs in length or more. We also trimmed the adapter sequence that was found in both read 1 (AGATCGGAAGAGCACACGTCTGAACTCCAGTCA) and read 2 (AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT) of all 16 samples. After this trim, we performed our FASTQC analysis again and found the quality scores to have improved from the first run-through. We used these trimmed reads for our next step; alignment.

To perform alignment of our reads to the reference genome, we utilized STAR (v. 2.7.10a) to perform a two-pass alignment. We set certain parameters for our base alignment:
- !STAR \
--runThreadN 16 \
--runMode genomeGenerate \
--genomeDir ../Sm_Mira_IvT/genome/star \
--genomeFastaFiles ../Sm_Mira_IvT/genome/genome.fa \
--sjdbGTFfile ../Sm_Mira_IvT/genome/annotations.gtf \
--sjdbOverhang 150 \
--genomeSAindexNbases 13

After this aligned, we took those reads to perform our first pass test, with parameters as:
- !STAR \
--runThreadN 16 \
--runMode alignReads \
--genomeDir ../Sm_Mira_IvT/genome/star \
--readFilesManifest manifest_mira.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outFileNamePrefix alignment_mira/star/first_pass/

We took the output of this first pass and performed our second pass test with parameters as: 
- !STAR \
--runThreadN 16 \
--runMode alignReads \
--genomeDir ../Sm_Mira_IvT/genome/star \
--readFilesManifest manifest_mira.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outSAMattributes NH HI AS nM RG \
--outFileNamePrefix alignment_mira/star/second_pass/ \
--sjdbFileChrStartEnd alignment_mira/star/SJ.out.tab

The output from our alignment passes gave us a .bam file that we used for alignment QC analysis. To do this, Samtools (v. 1.6), Picard (v. 3.2.0), and Qualimap (v. 2.3) were used. Samtools took the output .bam files from STAR and performed analysis to output a star_stats.txt file that we could use to view summaries associated with the entirety of our reads. We used Picard and Qualimap for our next step of analysis with parameters set as:

- !picard AddOrReplaceReadGroups I=alignment_mira/star/second_pass/Aligned.sortedByCoord.out.bam O=qualimap/output.bam RGLB=temp RGPL=Illumina RGPU=1 RGSM=20 VALIDATION_STRINGENCY=LENIENT

- !qualimap bamqc -nt 32 -outdir qualimap/star/bam -bam alignment_mira/star/second_pass/Aligned.sortedByCoord.out.bam --feature-file genome/annotations.gtf
- !qualimap rnaseq -outdir qualimap/star/rnaseq -bam alignment_mira/star/second_pass/Aligned.sortedByCoord.out.bam -gtf genome/annotations.gtf

After receiving the output files and viewing our analysis, we were able to step further and begin generating counts from our aligned reads. Using Picard tools once again, parameters were set as: 
- !picard MarkDuplicates I=alignment_mira/star/second_pass/Aligned.sortedByCoord.out.bam M=counting_mira/logs/star_duplicates O=counting_mira/dedup/star.bam VALIDATION_STRINGENCY=SILENT

This allowed us to locate duplicated reads and ensure that overrepresentation wasn't going to be influenced by these multiple mapped alignments. We further used FeatureCounts (v. 2.0.6) to get our counts from our .bam files. We set parameters as: 

- !featureCounts -T 32 \
    counting_mira/dedup/star.bam \
    -T 32 \
    -p \
    --byReadGroup \
    -s 1 \
    --ignoreDup \
    -M \
    --fraction \
    -a ../Sm_Mira_IvT/genome/annotations.gtf \
    -o counting_mira/star_counts.tsv \
    --verbose

We ran a multiqc report on the .tsv files output and were able to view now the entirety of our analysis and the quality of each step. This provided useful insight to our aligned reads and how well we had performed the steps. This set our foundation for identification of differentially expressed genes (DEGs) and the gene ontology (GO) behavior of these genes.

### Identification of differentially expressed genes and GO analysis

# Results

### RNA sequencing, alignment, and sample clustering

### Identification of differentially expressed genes

### Identification of pathways and/or GO annotations that are differentially expressed

# Discussion

# References